In [358]:

import warnings

from llama_index.core.agent import AgentOutput
from llama_index.core.async_utils import asyncio_run

warnings.filterwarnings("ignore")
from typing import List
from ultralytics.engine.results import Results
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings, SimpleDirectoryReader
from llama_index.core.tools import FunctionTool
from ultralytics import YOLO


async def main():
    # Initialize Ollama LLM
    llm = Ollama(model="llama3.1:latest", request_timeout=120.0)  # Adjust model and timeout as needed
    # Set as global LLM in LlamaIndex Settings
    Settings.llm = llm
    llm = Ollama(
        model="llama3.1:latest",
        base_url="http://localhost:11434/",
        request_timeout=360.0,
        # Manually set the context window to limit memory usage
        context_window=8000,
    )
    yolo_model = YOLO('predict/best.pt')

    def analyse_image_ppe_violation() -> str:
        image_directory_path = "./test"
        image_documents = SimpleDirectoryReader(image_directory_path).load_data()

        for doc in image_documents:
            yolo_responses: List[Results] = yolo_model.predict(doc.image_path, classes=[0, 1, 2, 3, 4, 6])
            if yolo_responses and len(yolo_responses) > 0:
                yolo_response = yolo_responses[0]
                detected_objects = []
                if yolo_response.boxes is not None:
                    detected_objects.append([yolo_model.names[int(c)] for c in yolo_response.boxes.cls])
                    doc.metadata = {'file_path': doc.image_path, 'detected_objects': detected_objects}
        return list(zip([os.path.split(img.image_path)[1] for img in image_documents],
                        [img.metadata['detected_objects'][0] for img in image_documents]))

    analyse_image_ppe_violation = FunctionTool.from_defaults(fn=analyse_image_ppe_violation)
    agent = FunctionAgent(
        tools=[analyse_image_ppe_violation],
        llm=llm,
        description="You are a Senior Analyst, who can analyse the json response and generate and return report"
    )
    response: AgentOutput = await agent.run(user_msg="Share analyse_image_ppe_violation details")
    print(response.response.content)


if __name__ == '__main__':
    asyncio_run(main())




2025-11-05 17:16:50,668 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"



image 1/1 /Users/venugopalgotagi/PycharmProjects/llamaindex_poc/test/1.png: 640x512 1 helmet, 2 Persons, 32.0ms
Speed: 1.8ms preprocess, 32.0ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 512)

image 1/1 /Users/venugopalgotagi/PycharmProjects/llamaindex_poc/test/2.png: 448x640 1 Person, 25.3ms
Speed: 1.0ms preprocess, 25.3ms inference, 0.4ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /Users/venugopalgotagi/PycharmProjects/llamaindex_poc/test/3.png: 256x640 7 helmets, 7 vests, 8 Persons, 15.2ms
Speed: 0.6ms preprocess, 15.2ms inference, 0.7ms postprocess per image at shape (1, 3, 256, 640)

image 1/1 /Users/venugopalgotagi/PycharmProjects/llamaindex_poc/test/4.png: 640x640 1 helmet, 1 vest, 1 Person, 28.4ms
Speed: 1.2ms preprocess, 28.4ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /Users/venugopalgotagi/PycharmProjects/llamaindex_poc/test/5.png: 544x640 3 helmets, 3 vests, 3 Persons, 28.6ms
Speed: 1.0ms preprocess, 28.6ms

2025-11-05 17:16:52,105 - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


The `analyse_image_ppe_violation` tool is used to identify potential Personal Protective Equipment (PPE) violations in images. The output of this tool is a list of tuples, where each tuple contains the image name and a list of detected objects or PPEs.

In this case, the tool has analyzed 7 different images:

1. `1.png`: A person wearing a helmet.
2. `2.png`: A single person in the image, but no PPE is detected.
3. `3.png`: Multiple people are present, some of whom are wearing helmets and vests.
4. `4.png`: A person wearing a vest and helmet.
5. `5.png`: Some people are wearing helmets, while others have no PPE on.
6. `6.png`: A single person in the image, but no PPE is detected.
7. `7.png`: A person wearing a helmet.

The tool has identified some potential PPE violations:

* In image `3.png`, there are multiple people without PPE.
* In image `5.png`, there are people without PPE.
* In image `6.png`, there is no one wearing any PPE.
